In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [2]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb.xlsx', 'jan.xlsx']
sheet = 'Sheet'

In [3]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [4]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [80]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[(df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')) & (df.inn != 9718071371)]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_x', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [81]:
jan = load_data('jan')
feb = load_data('feb')

In [82]:
res = pd.merge(jan, feb, how='outer', on=['inn', 'snils'])

In [20]:
def szp_teach(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0:
        return np.nan
    return sum_szp / cnt

In [21]:
def szp_teach_norm(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month] / row['stv_' + month]
    if cnt == 0:
        return np.nan
    return sum_szp / cnt

In [23]:
def szp_teach_norm_sub1(row , months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            if row['stv_' + month] < 1:
                sum_szp += row['sum_' + month] / row['stv_' + month]
                continue
            sum_szp += row['sum_' + month]
    if cnt == 0:
        return np.nan
    return sum_szp / cnt

In [83]:
res['szp1'] = res.apply(lambda row: szp_teach(row, ['jan', 'feb']), axis=1)
res['szp2'] = res.apply(lambda row: szp_teach_norm(row, ['jan', 'feb']), axis=1)
res['szp3'] = res.apply(lambda row: szp_teach_norm_sub1(row, ['jan', 'feb']), axis=1)

In [84]:
res[res.inn == 7737027335].groupby('inn').count()

,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,szp1,szp2,szp3
inn,,,,,,,,,,
7737027335,563,504,504,504,500,500,500,316,316,316


In [85]:
calc = res[['inn', 'szp1', 'szp2', 'szp3']].groupby('inn').mean()
calc[calc.index == 7737027335]

,szp1,szp2,szp3
inn,,,
7737027335,120164.442658,89596.426486,124237.051595


In [31]:
print_df(calc, 'szp_diff')